In [24]:

%%configure -f
{
  "conf": {
    "spark.jars": "abfss://f7e2db24-d041-47bb-ae02-0c67982c6c4f@onelake.dfs.fabric.microsoft.com/8fcfa082-2992-46d9-9794-dea041d8c11a/Files/Drivers/neo4j-spark-connector-5.3.8-s_2.12.jar"
  }
}



StatementMeta(, fabdcf33-45e0-4bb3-a679-7fa46ec9fe0e, -1, Finished, Available, Finished)

In [25]:
%%spark
//JSON File in lakehouse with Neo4j credentials
val absfss_JSON_Base_Path = "abfss://f7e2db24-d041-47bb-ae02-0c67982c6c4f@onelake.dfs.fabric.microsoft.com/8fcfa082-2992-46d9-9794-dea041d8c11a/Files/Drivers/neo4j-conn.json"

// Import required libraries
import org.apache.spark.sql.{DataFrame, SaveMode, SparkSession}
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import com.fasterxml.jackson.module.scala.experimental.ScalaObjectMapper

// Load JSON file for Neo4j credentials
val jsonString = spark.read.text(absfss_JSON_Base_Path).as[String].collect().mkString("\n")

// Parse JSON string
val mapper = new ObjectMapper() with ScalaObjectMapper
mapper.registerModule(DefaultScalaModule)
val data = mapper.readValue[Map[String, Any]](jsonString)

// Extract Neo4j connection details
val neo4jUrl = data("neo4j-url").asInstanceOf[String]
val neo4jUsername = data("neo4j-username").asInstanceOf[String]
val neo4jPassword = data("neo4j-password").asInstanceOf[String]

val spark = SparkSession.builder 
    .config("spark.neo4j.bolt.url", neo4jUrl) 
    .config("spark.neo4j.bolt.user", neo4jUsername)
    .config("spark.neo4j.bolt.password", neo4jPassword) 
    .getOrCreate()


StatementMeta(, fabdcf33-45e0-4bb3-a679-7fa46ec9fe0e, 3, Finished, Available, Finished)

absfss_JSON_Base_Path: String = abfss://f7e2db24-d041-47bb-ae02-0c67982c6c4f@onelake.dfs.fabric.microsoft.com/8fcfa082-2992-46d9-9794-dea041d8c11a/Files/Drivers/neo4j-conn.json
import org.apache.spark.sql.{DataFrame, SaveMode, SparkSession}
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import com.fasterxml.jackson.module.scala.experimental.ScalaObjectMapper
jsonString: String =
{
  "neo4j-url": "neo4j+s://a2e6569d.databases.neo4j.io",
  "neo4j-username": "neo4j",
  "neo4j-password": "CnpwsEa4_OO3BQFlbJiZ6j8Ipi30C-kQjUwM4KHabU0"
}
mapper: com.fasterxml.jackson.databind.ObjectMapper with com.fasterxml.jackson.module.scala.experimental.ScalaObjectMapper = $default$anon$1@2f1f120e
res12: com.fasterxml.jackson.databind.ObjectMapper = $default$anon$1@2f1f120e
data: Map[String,Any] = Map(neo4j-url -> neo4j+s://a2e6569d.databases.neo4j.io, neo4j-username -> neo4j, neo4j-password -> CnpwsEa4_OO3BQFlbJiZ6j8Ipi30C-kQjUwM4KHabU0)
ne

In [27]:
%%spark

val df = spark.read.format("org.neo4j.spark.DataSource")  
    .option("query", "match(o:Order)-[oc:ORDER_CONTAINS]->(p:Product) WHERE p.productID = \"41\" RETURN o.customerID, o.employeeID, o.freight, o.orderDate, o.orderID, o.requiredDate, o.shippedDate, p.productID") 
    .option("url", neo4jUrl)
    .option("authentication.basic.username", neo4jUsername)
    .option("authentication.basic.password", neo4jPassword)
    .load()

df.show()

StatementMeta(, fabdcf33-45e0-4bb3-a679-7fa46ec9fe0e, 5, Finished, Available, Finished)

+------------+------------+---------+--------------------+---------+--------------------+--------------------+-----------+
|o.customerID|o.employeeID|o.freight|         o.orderDate|o.orderID|      o.requiredDate|       o.shippedDate|p.productID|
+------------+------------+---------+--------------------+---------+--------------------+--------------------+-----------+
|       HANAR|           4|    65.83|1996-07-08 00:00:...|    10250|1996-08-05 00:00:...|1996-07-12 00:00:...|         41|
|       OTTIK|           4|    55.09|1996-07-19 00:00:...|    10260|1996-08-16 00:00:...|1996-07-29 00:00:...|         41|
|       FOLKO|           6|     3.67|1996-07-24 00:00:...|    10264|1996-08-21 00:00:...|1996-08-23 00:00:...|         41|
|       RATTC|           1|   150.15|1996-09-27 00:00:...|    10316|1996-10-25 00:00:...|1996-10-08 00:00:...|         41|
|       ISLAT|           8|     4.73|1996-10-01 00:00:...|    10318|1996-10-29 00:00:...|1996-10-04 00:00:...|         41|
|       BONAP|  

In [28]:
%%spark
//Write to Files/
df.write.mode("overwrite").format("csv").option("header", "true").save("Files/ExportFromNeo4j/test.csv")


StatementMeta(, fabdcf33-45e0-4bb3-a679-7fa46ec9fe0e, 6, Finished, Available, Finished)

In [29]:
   %%spark
   //Write to Tables/
   df.write.format("delta").mode("overwrite").saveAsTable("Lakehouse1.Analysis")

StatementMeta(, fabdcf33-45e0-4bb3-a679-7fa46ec9fe0e, 7, Finished, Available, Finished)